In [1]:
import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy

In [2]:
class Sphere:
    def __init__(self, x_0, y_0, z_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 =z_0
        self.r = r
        
    def distance(self, particle):
        x_ = particle.coordinates.x - self.x_0
        y_ = particle.coordinates.y - self.y_0
        z_ = particle.coordinates.z - self.z_0
        
        c = x_ * x_ + y_ * y_ + z_ * z_ - self.r * self.r
        
        k = x_ * particle.direction.tetta_x + y_ * particle.direction.tetta_y + z_ * particle.direction.tetta_z
        
        dec = k * k - c
        if dec < 0:
            return -1
        
        dist_1 = (-k + math.sqrt(k * k - c))
        dist_2 = (-k - math.sqrt(k * k - c))
         
        distance = min(dist_1, dist_2)
        
        return distance
     
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y)  + 
                (self.z_0 - particle.coordinates.z) * (self.z_0 - particle.coordinates.z) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1  
        
    def get_xml(self):

        obj_xml = " Sphere with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " z_0 " + str(z_0) + " r = "+ str(r)
        
        return obj_xml

In [3]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]

energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  

In [4]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [5]:
class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


In [6]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [7]:
import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [8]:
def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [9]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    

In [10]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [11]:
def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = 0.
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False

In [12]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)

In [13]:
def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision
        

In [14]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [15]:
def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        current_xs = universe.get_xs_by_coordinates(particle)
        free_path = get_free_path(particle, current_xs)
        set_random_direction(particle)
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)

        if is_collision_virual(particle, current_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions

In [16]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [17]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])

In [18]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))

In [19]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [20]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])

In [21]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [22]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  

In [23]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std    

In [24]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))

In [25]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      
        batch_particles = normalise_weights(batch_particles, initial_size) 

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators

In [26]:
cs_fission_u235_b = [0.065280]
cs_capture_u235_b = [0.013056]
cs_scattering_u235_b = [0.248064]
cs_total_u235_b = [0.32640]
cs_production_neutrons_u235_b = [2.797101]


cs_fission_h2o = [0.0]
cs_capture_h2o = [0.032640]
cs_scattering_h2o = [0.293760]
cs_production_neutrons_h2o = [0.0]
cs_total_h2o = [0.32640]





u235_b = Sig(1, cs_fission_u235_b, cs_capture_u235_b, cs_scattering_u235_b, 
                       cs_production_neutrons_u235_b,  cs_total_u235_b)

h2o = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]

In [27]:
test_number_of_particles = 10
test_number_interations = 500
test_number_inactive = 200

In [28]:
sphere = Sphere(0, 0, 0, 6.12745)
boundaries_type = ["black"]

surfaces = [sphere]
signs = [-1]

sphere_u235_b = Cell(surfaces, signs)
sphere_u235_b.set_boundaries_type(boundaries_type)

In [29]:
sphere_in = Sphere(0, 0, 0, 6.12745)
sphere_out = Sphere(0, 0, 0, 9.191176)
boundaries_type = ["black", "black"]

surfaces = [sphere_in, sphere_out]
signs = [+1, -1]

sphere_h2o = Cell(surfaces, signs)
sphere_h2o.set_boundaries_type(boundaries_type)

In [30]:
cells = [sphere_u235_b, sphere_h2o]
materials = [u235_b, h2o]

In [31]:
water_sphere_universe = Universe(cells, materials)
estimators = []

In [32]:
test_number_of_particles = 20
test_number_interations = 500
test_number_inactive = 200

In [33]:
k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(water_sphere_universe,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, u235_b, estimators)

 num1  8
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
i == 103
i == 104
i == 105
i == 106
i == 107
i == 108
i == 109
i == 110
i == 111
i == 112
i == 113
i == 114
i == 115
i == 116
i == 117
i == 118
i == 119
i == 120
i == 121
i =

 keff_cycle , k_effective_exp, std_k_effective 1.0327040127811418   1.0003965508987342  0.0018430107087872008
i == 258
 keff_cycle , k_effective_exp, std_k_effective 0.994889741973308   1.0003016059172614  0.0018134430880226821
i == 259
 keff_cycle , k_effective_exp, std_k_effective 1.008871604867231   1.0004468601367524  0.0017883504899428894
i == 260
 keff_cycle , k_effective_exp, std_k_effective 0.9881292870429002   1.000241567251855  0.001770236159661349
i == 261
 keff_cycle , k_effective_exp, std_k_effective 1.0119237433341453   1.0004330783351711  0.0017514756953447763
i == 262
 keff_cycle , k_effective_exp, std_k_effective 0.9992651767989332   1.000414241213619  0.0017230974856167675
i == 263
 keff_cycle , k_effective_exp, std_k_effective 0.9886943777958539   1.0002282116355592  0.0017057010051160378
i == 264
 keff_cycle , k_effective_exp, std_k_effective 1.0187735093997978   1.0005179819131254  0.0017036617871955947
i == 265
 keff_cycle , k_effective_exp, std_k_effective 0.9846

 keff_cycle , k_effective_exp, std_k_effective 0.9844321743001948   1.0015416776141022  0.0013024289694201967
i == 328
 keff_cycle , k_effective_exp, std_k_effective 0.9927125288678724   1.0014726998895223  0.001294053371312275
i == 329
 keff_cycle , k_effective_exp, std_k_effective 1.0028094360253805   1.0014830621851492  0.0012840245774115675
i == 330
 keff_cycle , k_effective_exp, std_k_effective 0.9948489053242109   1.0014320302092956  0.0012751307652506892
i == 331
 keff_cycle , k_effective_exp, std_k_effective 1.0074870714981372   1.0014782518985235  0.0012662034274285615
i == 332
 keff_cycle , k_effective_exp, std_k_effective 1.0131030018696958   1.0015663181861838  0.0012596566214619885
i == 333
 keff_cycle , k_effective_exp, std_k_effective 0.9876425890380944   1.001461628493341  0.001254525432996992
i == 334
 keff_cycle , k_effective_exp, std_k_effective 0.9864390848502147   1.001349519958691  0.0012501649191007953
i == 335
 keff_cycle , k_effective_exp, std_k_effective 1.005

 keff_cycle , k_effective_exp, std_k_effective 1.0142317585126386   1.0004863046256778  0.0010709148119843117
i == 398
 keff_cycle , k_effective_exp, std_k_effective 1.004734045583952   1.0005077578628407  0.0010657083776861637
i == 399
 keff_cycle , k_effective_exp, std_k_effective 0.9868449137941664   1.000439100354958  0.0010625600143609505
i == 400
 keff_cycle , k_effective_exp, std_k_effective 0.9904891828709135   1.0003893507675377  0.0010584037356916492
i == 401
 keff_cycle , k_effective_exp, std_k_effective 0.9988313701150326   1.0003815996200127  0.001053153405539196
i == 402
 keff_cycle , k_effective_exp, std_k_effective 0.9726269440716305   1.0002442003351197  0.0010568959992311884
i == 403
 keff_cycle , k_effective_exp, std_k_effective 0.9985024005392602   1.0002356200405589  0.0010517117291889988
i == 404
 keff_cycle , k_effective_exp, std_k_effective 0.9884850055854454   1.0001780189893081  0.0010481275440901807
i == 405
 keff_cycle , k_effective_exp, std_k_effective 0.99

 keff_cycle , k_effective_exp, std_k_effective 0.97882562492229   1.0005333753202563  0.0009349196716964252
i == 468
 keff_cycle , k_effective_exp, std_k_effective 1.0028280292043041   1.0005419374616147  0.0009314639856573736
i == 469
 keff_cycle , k_effective_exp, std_k_effective 1.0245961248023028   1.000631358232398  0.0009322931331751954
i == 470
 keff_cycle , k_effective_exp, std_k_effective 0.9793046944562546   1.0005523705887824  0.0009321862657816863
i == 471
 keff_cycle , k_effective_exp, std_k_effective 0.9977672506956979   1.0005420933936051  0.0009287969546753775
i == 472
 keff_cycle , k_effective_exp, std_k_effective 1.0105009047083977   1.0005787066704976  0.000926099994065163
i == 473
 keff_cycle , k_effective_exp, std_k_effective 0.9991256592838762   1.000573384152598  0.0009227168016120259
i == 474
 keff_cycle , k_effective_exp, std_k_effective 1.014735398183334   1.000625070335192  0.0009207948267405047
i == 475
 keff_cycle , k_effective_exp, std_k_effective 1.023764

In [151]:
difference = (1.  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  97.70714760209964  with standart deviation [pcm]+- 84.50380226736411


In [18]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import matplotlib.pyplot as plt

import math
import random
import h5py
import copy


# In[2]:


class Sphere:
    def __init__(self, x_0, y_0, z_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 =z_0
        self.r = r
        
    def distance(self, particle):
        x_ = particle.coordinates.x - self.x_0
        y_ = particle.coordinates.y - self.y_0
        z_ = particle.coordinates.z - self.z_0
        
        c = x_ * x_ + y_ * y_ + z_ * z_ - self.r * self.r
        
        k = x_ * particle.direction.tetta_x + y_ * particle.direction.tetta_y + z_ * particle.direction.tetta_z
        
        dec = k * k - c
        if dec < 0:
            return -1
        
        dist_1 = (-k + math.sqrt(k * k - c))
        dist_2 = (-k - math.sqrt(k * k - c))
         
        distance = min(dist_1, dist_2)
        
        return distance
     
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y)  + 
                (self.z_0 - particle.coordinates.z) * (self.z_0 - particle.coordinates.z) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1  
        
    def get_xml(self):

        obj_xml = " Sphere with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " z_0 " + str(z_0) + " r = "+ str(r)
        
        return obj_xml


# In[3]:


class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]

energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  


# In[4]:


class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        


# In[8]:


class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


# In[9]:


def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))


# In[10]:


import random

def make_initial_sources(number_of_paricles, box_size, energy=10.0e6):
    
    step_x = 2 * box_size[0]/number_of_paricles
    step_y = 2 * box_size[1]/number_of_paricles
    step_z = 2 * box_size[2]/number_of_paricles
    
    x_coord = -box_size[0]
    y_coord = -box_size[1]
    z_coord = -box_size[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources


# In[11]:


def get_free_path(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = c_s.sig_t[energy_group_idx]
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  


# In[12]:


class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    


# In[13]:


def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)


# In[14]:


def is_collision_virual(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = 0.
    total_cs = c_s.sig_t[energy_group_idx]
    if virtual_cs / total_cs >= random_number:
        return True
    else:
        return False


# In[15]:


def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)


# In[16]:


def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision
        


# In[17]:


def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles


# In[18]:


def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    while not particle.is_terminated():
        current_xs = universe.get_xs_by_coordinates(particle)
        free_path = get_free_path(particle, current_xs)
        set_random_direction(particle)
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)

        if is_collision_virual(particle, current_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions


# In[19]:


def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights


# In[20]:


def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])


# In[21]:


def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))


# In[22]:


def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles


# In[23]:


def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])


# In[24]:


def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")


# In[25]:


def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  


# In[26]:


def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std    


# In[27]:


def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))


# In[28]:


import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    initial_sources = make_initial_sources(number_of_particles, energy=10.0e6, box_size=[0.5, 0.5, 0.5])
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      
        batch_particles = normalise_weights(batch_particles, initial_size) 

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators


# In[29]:


cs_fission_u235_b = [0.065280]
cs_capture_u235_b = [0.013056]
cs_scattering_u235_b = [0.248064]
cs_total_u235_b = [0.32640]
cs_production_neutrons_u235_b = [2.797101]


cs_fission_h2o = [0.0]
cs_capture_h2o = [0.032640]
cs_scattering_h2o = [0.293760]
cs_production_neutrons_h2o = [0.0]
cs_total_h2o = [0.32640]





u235_b = Sig(1, cs_fission_u235_b, cs_capture_u235_b, cs_scattering_u235_b, 
                       cs_production_neutrons_u235_b,  cs_total_u235_b)

h2o = Sig(1, cs_fission_h2o, cs_capture_h2o, cs_scattering_h2o, cs_production_neutrons_h2o,
                cs_total_h2o)



energy_groups_2 = [0, 2 * 1E10]


# In[34]:


test_number_of_particles = 10
test_number_interations = 500
test_number_inactive = 200


# In[35]:


sphere = Sphere(0, 0, 0, 6.12745)
boundaries_type = ["black"]

surfaces = [sphere]
signs = [-1]

sphere_u235_b = Cell(surfaces, signs)
sphere_u235_b.set_boundaries_type(boundaries_type)


# In[36]:


sphere_in = Sphere(0, 0, 0, 6.12745)
sphere_out = Sphere(0, 0, 0, 9.191176)
boundaries_type = ["black", "black"]

surfaces = [sphere_in, sphere_out]
signs = [+1, -1]

sphere_h2o = Cell(surfaces, signs)
sphere_h2o.set_boundaries_type(boundaries_type)


# In[37]:


cells = [sphere_u235_b, sphere_h2o]
materials = [u235_b, h2o]


# In[38]:


water_sphere_universe = Universe(cells, materials)
estimators = []


# In[39]:


test_number_of_particles = 10
test_number_interations = 1000
test_number_inactive = 200


# In[ ]:


k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(water_sphere_universe,  test_number_interations,  test_number_inactive, 
                           test_number_of_particles, u235_b, estimators)


# In[151]:


difference = (1.  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


# In[ ]:




def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()


# In[58]:


file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)

 num1  4
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
i == 103
i == 104
i == 105
i == 106
i == 107
i == 108
i == 109
i == 110
i == 111
i == 112
i == 113
i == 114
i == 115
i == 116
i == 117
i == 118
i == 119
i == 120
i == 121
i =

 keff_cycle , k_effective_exp, std_k_effective 0.9313893029961664   0.9949245524053585  0.006151244680702751
i == 258
 keff_cycle , k_effective_exp, std_k_effective 1.0457141489573052   0.99580023510453  0.006107362704461832
i == 259
 keff_cycle , k_effective_exp, std_k_effective 0.9197136388229974   0.9945106317777244  0.006139914759952551
i == 260
 keff_cycle , k_effective_exp, std_k_effective 1.0272870826954343   0.9950569059596862  0.006061381880345314
i == 261
 keff_cycle , k_effective_exp, std_k_effective 1.0360783514932868   0.9957293886733518  0.005998998387264125
i == 262
 keff_cycle , k_effective_exp, std_k_effective 1.0054499956230918   0.9958861726564121  0.005903529472600101
i == 263
 keff_cycle , k_effective_exp, std_k_effective 1.0127982316910364   0.996154618037914  0.005815266223013508
i == 264
 keff_cycle , k_effective_exp, std_k_effective 1.0383841299304022   0.9968144541612342  0.00576158950857041
i == 265
 keff_cycle , k_effective_exp, std_k_effective 1.04692005259

 keff_cycle , k_effective_exp, std_k_effective 1.043721142978649   0.995312552805652  0.0038581783559298124
i == 329
 keff_cycle , k_effective_exp, std_k_effective 0.9968597392955456   0.9953245464993721  0.0038281719534859465
i == 330
 keff_cycle , k_effective_exp, std_k_effective 1.1460185952864772   0.9964837314900421  0.0039715425643287346
i == 331
 keff_cycle , k_effective_exp, std_k_effective 1.009285320611511   0.9965814535444046  0.003942320190523208
i == 332
 keff_cycle , k_effective_exp, std_k_effective 0.9743249191317109   0.9964128434352175  0.003915971743344088
i == 333
 keff_cycle , k_effective_exp, std_k_effective 0.9733960759613844   0.9962397850331585  0.0038902679711335523
i == 334
 keff_cycle , k_effective_exp, std_k_effective 0.9686346648148179   0.9960337766733203  0.0038666188039455183
i == 335
 keff_cycle , k_effective_exp, std_k_effective 1.0121886976312433   0.99615344275449  0.003839735470982129
i == 336
 keff_cycle , k_effective_exp, std_k_effective 0.9637911

 keff_cycle , k_effective_exp, std_k_effective 0.9848104568173116   0.9943394879463853  0.003135600332544309
i == 400
 keff_cycle , k_effective_exp, std_k_effective 1.0025948969491956   0.9943807649913994  0.0031201559809475154
i == 401
 keff_cycle , k_effective_exp, std_k_effective 0.9414209965415113   0.9941172835563253  0.003115754551557203
i == 402
 keff_cycle , k_effective_exp, std_k_effective 0.978089566237809   0.9940379384210851  0.0031013068201347685
i == 403
 keff_cycle , k_effective_exp, std_k_effective 0.9446041032716193   0.9937944219917774  0.0030955846941587263
i == 404
 keff_cycle , k_effective_exp, std_k_effective 0.9860537872831198   0.9937564777039899  0.0030806065752737237
i == 405
 keff_cycle , k_effective_exp, std_k_effective 0.9109162125437343   0.9933523788495495  0.0030920617805265636
i == 406
 keff_cycle , k_effective_exp, std_k_effective 1.0051504258920698   0.9934096509225715  0.0030775481144636047
i == 407
 keff_cycle , k_effective_exp, std_k_effective 1.07

 keff_cycle , k_effective_exp, std_k_effective 1.0183208682103564   0.9965705794254955  0.0026699120944225635
i == 471
 keff_cycle , k_effective_exp, std_k_effective 1.0007759125870233   0.99658609726004  0.0026600870382904474
i == 472
 keff_cycle , k_effective_exp, std_k_effective 1.1221358302210003   0.9970476771606317  0.002690183856269694
i == 473
 keff_cycle , k_effective_exp, std_k_effective 1.011397209965675   0.9971002395518589  0.002680826927862008
i == 474
 keff_cycle , k_effective_exp, std_k_effective 1.0378332308386158   0.9972489001040002  0.002675158750756929
i == 475
 keff_cycle , k_effective_exp, std_k_effective 0.9642583418074077   0.9971289344374672  0.0026681115075604064
i == 476
 keff_cycle , k_effective_exp, std_k_effective 0.9162495996833236   0.9968358933695175  0.0026745291995475236
i == 477
 keff_cycle , k_effective_exp, std_k_effective 0.9694947118443661   0.9967371887430729  0.002666683720393795
i == 478
 keff_cycle , k_effective_exp, std_k_effective 1.034137

...

 keff_cycle , k_effective_exp, std_k_effective 1.0197466590916604   0.9975297209689553  0.0020103494052744223
i == 683
 keff_cycle , k_effective_exp, std_k_effective 1.0827350683381076   0.9977061295556409  0.002013923958719483
i == 684
 keff_cycle , k_effective_exp, std_k_effective 0.998649790093621   0.9977080792674963  0.002009759597076164
i == 685
 keff_cycle , k_effective_exp, std_k_effective 0.9558378835269068   0.9976217489670002  0.0020074686392658027
i == 686
 keff_cycle , k_effective_exp, std_k_effective 1.0287065714601962   0.9976857095071096  0.0020043545626618224
i == 687
 keff_cycle , k_effective_exp, std_k_effective 0.9783594754304634   0.9976460252482253  0.002000628235783256
i == 688
 keff_cycle , k_effective_exp, std_k_effective 1.0614123770984554   0.9977766940020169  0.0020007958211279023
i == 689
 keff_cycle , k_effective_exp, std_k_effective 0.9751540620400646   0.9977304309509699  0.0019972359017842063
i == 690
 keff_cycle , k_effective_exp, std_k_effective 0.970

 keff_cycle , k_effective_exp, std_k_effective 0.9865179658191526   0.998568977335521  0.0018949870803441895
i == 754
 keff_cycle , k_effective_exp, std_k_effective 1.0217829651491082   0.9986108798405996  0.0018920274950971852
i == 755
 keff_cycle , k_effective_exp, std_k_effective 1.0165078310164364   0.998643126599475  0.0018888906349084228
i == 756
 keff_cycle , k_effective_exp, std_k_effective 0.9662901560701985   0.9985849378035588  0.0018863879679920665
i == 757
 keff_cycle , k_effective_exp, std_k_effective 0.9304070727797087   0.9984625358914874  0.0018869723263033346
i == 758
 keff_cycle , k_effective_exp, std_k_effective 1.053247001700054   0.9985607159377392  0.0018861446506430771
i == 759
 keff_cycle , k_effective_exp, std_k_effective 1.049200086414559   0.9986513051514724  0.0018849455732023329
i == 760
 keff_cycle , k_effective_exp, std_k_effective 1.040322697945322   0.9987257183528898  0.0018830474720521375
i == 761
 keff_cycle , k_effective_exp, std_k_effective 0.9859

 keff_cycle , k_effective_exp, std_k_effective 0.9208622707506746   0.9987054902399773  0.0017871808408870171
i == 825
 keff_cycle , k_effective_exp, std_k_effective 0.963340176030404   0.9986489057372421  0.0017852160415024621
i == 826
 keff_cycle , k_effective_exp, std_k_effective 0.9842663824136761   0.9986259304603674  0.0017825100508363687
i == 827
 keff_cycle , k_effective_exp, std_k_effective 1.0462686517910675   0.9987019156618517  0.0017812862683232312
i == 828
 keff_cycle , k_effective_exp, std_k_effective 1.098079482567429   0.9988601601951408  0.0017854739056436353
i == 829
 keff_cycle , k_effective_exp, std_k_effective 0.9578897146231371   0.998795024351624  0.0017838226606309029
i == 830
 keff_cycle , k_effective_exp, std_k_effective 1.0378122093642201   0.9988569563913265  0.0017820654311503658
i == 831
 keff_cycle , k_effective_exp, std_k_effective 0.9950222045811468   0.9988508791301377  0.0017792493761582965
i == 832
 keff_cycle , k_effective_exp, std_k_effective 0.96

 keff_cycle , k_effective_exp, std_k_effective 0.9841568674229035   0.9984871110576777  0.0016603880629984464
i == 896
 keff_cycle , k_effective_exp, std_k_effective 0.9614442951068862   0.9984338886209667  0.0016588547417027287
i == 897
 keff_cycle , k_effective_exp, std_k_effective 0.9726775922453981   0.9983969355415183  0.00165688516820245
i == 898
 keff_cycle , k_effective_exp, std_k_effective 1.017044260695799   0.9984236509070689  0.0016547253767996552
i == 899
 keff_cycle , k_effective_exp, std_k_effective 0.9315356736314284   0.9983279599524542  0.0016551249048541304
i == 900
 keff_cycle , k_effective_exp, std_k_effective 1.009165189385558   0.9983434417087873  0.0016528312583097035
i == 901
 keff_cycle , k_effective_exp, std_k_effective 1.0728180470639668   0.9984496822299787  0.0016538875603385056
i == 902
 keff_cycle , k_effective_exp, std_k_effective 0.9898819322687747   0.9984374774579541  0.0016515750106114743
i == 903
 keff_cycle , k_effective_exp, std_k_effective 1.067

 keff_cycle , k_effective_exp, std_k_effective 1.0234810773648153   0.998749283278069  0.0015744425668144791
i == 966
 keff_cycle , k_effective_exp, std_k_effective 1.0091859966001289   0.9987629082301865  0.0015724448453485645
i == 967
 keff_cycle , k_effective_exp, std_k_effective 1.0006534545108385   0.9987653730884403  0.0015703953178846934
i == 968
 keff_cycle , k_effective_exp, std_k_effective 0.9738488442148059   0.9987329296914694  0.001568684730935523
i == 969
 keff_cycle , k_effective_exp, std_k_effective 1.0208218446250046   0.9987616538981451  0.0015669068059397243
i == 970
 keff_cycle , k_effective_exp, std_k_effective 1.1281150321414777   0.9989296452984612  0.001573861783694861
i == 971
 keff_cycle , k_effective_exp, std_k_effective 0.9909291631262502   0.9989192685381859  0.0015718533849140475
i == 972
 keff_cycle , k_effective_exp, std_k_effective 1.025196232620996   0.9989533060564277  0.0015701849506861666
i == 973
 keff_cycle , k_effective_exp, std_k_effective 1.042